In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum

spark = SparkSession.builder.appName("HogwartsAnalysis").getOrCreate()

data1 = [
    ("Gryffindor", 1, 80, "Harry Potter"),
    ("Slytherin", 1, 60, "Draco Malfoy"),
    ("Ravenclaw", 1, 45, "Luna Lovegood"),
    ("Hufflepuff", 1, 30, "Cedric Diggory"),
    ("Gryffindor", 2, 90, "Hermione Granger"),
    ("Slytherin", 2, 70, "Pansy Parkinson"),
    ("Ravenclaw", 2, 55, "Cho Chang"),
    ("Hufflepuff", 2, 65, "Hannah Abbott"),
    ("Gryffindor", 3, 20, "Ron Weasley"),
    ("Slytherin", 3, 85, "Blaise Zabini")
]
columns1 = ["house", "year", "points", "student"]
df1 = spark.createDataFrame(data1, columns1)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/11 09:04:15 WARN Utils: Your hostname, codespaces-7972f7, resolves to a loopback address: 127.0.0.1; using 10.0.2.146 instead (on interface eth0)
25/08/11 09:04:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/11 09:04:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
result1 = (
    df1.select("house", "year", "points")
       .filter(col("points") > 50)
       .groupBy("house", "year")
       .agg(_sum("points").alias("total_points"))
       .orderBy("year", col("total_points").desc())
)
result1.show()

+----------+----+------------+
|     house|year|total_points|
+----------+----+------------+
|Gryffindor|   1|          80|
| Slytherin|   1|          60|
|Gryffindor|   2|          90|
| Slytherin|   2|          70|
|Hufflepuff|   2|          65|
| Ravenclaw|   2|          55|
| Slytherin|   3|          85|
+----------+----+------------+



In [4]:
from pyspark.sql.functions import avg

data2 = [
    ("Aragorn", "Human", 10, 2, "Helms Deep"),
    ("Legolas", "Elf", 15, 0, "Helms Deep"),
    ("Gimli", "Dwarf", 8, 3, "Helms Deep"),
    ("Frodo", "Hobbit", 2, 1, "Moria"),
    ("Sam", "Hobbit", 4, 2, "Moria"),
    ("Gandalf", "Wizard", 12, 1, "Moria"),
    ("Boromir", "Human", 7, 4, "Amon Hen"),
    ("Legolas", "Elf", 20, 0, "Amon Hen"),
    ("Aragorn", "Human", 9, 2, "Amon Hen")
]
columns2 = ["name", "race", "enemies_defeated", "injuries", "battle"]
df2 = spark.createDataFrame(data2, columns2)

In [5]:
result2 = (
    df2.select("name", "race", "enemies_defeated")
       .filter(col("enemies_defeated") > 5)
       .groupBy("race")
       .agg(avg("enemies_defeated").alias("avg_enemies_defeated"))
       .orderBy(col("avg_enemies_defeated").desc())
)
result2.show()

+------+--------------------+
|  race|avg_enemies_defeated|
+------+--------------------+
|   Elf|                17.5|
|Wizard|                12.0|
| Human|   8.666666666666666|
| Dwarf|                 8.0|
+------+--------------------+



In [7]:
from pyspark.sql.functions import count, sum as _sum
data3 = [
    ("SUB-01", "Pacific Strike", 5, "Success"),
    ("SUB-02", "Atlantic Surge", 2, "Failure"),
    ("SUB-01", "Arctic Blitz", 4, "Success"),
    ("SUB-03", "Indian Ocean", 6, "Success"),
    ("SUB-02", "Pacific Strike", 3, "Success"),
    ("SUB-01", "Coral Sea", 7, "Success"),
    ("SUB-03", "Arctic Blitz", 1, "Failure"),
    ("SUB-02", "Bering Strait", 5, "Success")
]
columns3 = ["submarine_id", "mission_name", "warheads_launched", "status"]
df3 = spark.createDataFrame(data3, columns3)

In [8]:
result3 = (
    df3.select("submarine_id", "mission_name", "warheads_launched")
       .filter(col("warheads_launched") > 3)
       .groupBy("submarine_id")
       .agg(
            count("mission_name").alias("total_missions"),
            _sum("warheads_launched").alias("total_warheads_launched")
       )
       .orderBy(col("total_warheads_launched").desc())
)
result3.show()

+------------+--------------+-----------------------+
|submarine_id|total_missions|total_warheads_launched|
+------------+--------------+-----------------------+
|      SUB-01|             3|                     16|
|      SUB-03|             1|                      6|
|      SUB-02|             1|                      5|
+------------+--------------+-----------------------+

